# Lab 1 - Creating the SQL Tables

In this lab, use `sqlalchemy` to create, populate, and query a table from the baseball database, as well as for the `super_hero_powers.csv` table.  

In [1]:
import pandas as pd
artwork = pd.read_csv("./data/Artworks.csv")

## Part 1 - Baseball Managers

In this part of the lab, you will walk through the process of creating a manager table from [Lahman’s Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/)

## Task 1 - Download, unzip, rename 

1. Download the baseball database linked above (save to desktop)
2. Unzip the file and rename to `baseball`
3. Load the `core/Managers.csv` file into a pandas `DataFrame` using `read_csv`
4. Inspect the `column` names and `dtypes`

In [2]:
import pandas as pd
managers = pd.read_csv('~/Desktop/baseball/core/Managers.csv')
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y


In [3]:
managers.columns

Index(['playerID', 'yearID', 'teamID', 'lgID', 'inseason', 'G', 'W', 'L',
       'rank', 'plyrMgr'],
      dtype='object')

**Question:** Is there a candidate for a primary key?

In [4]:
[(col, managers[col].is_unique) for col in managers]

[('playerID', False),
 ('yearID', False),
 ('teamID', False),
 ('lgID', False),
 ('inseason', False),
 ('G', False),
 ('W', False),
 ('L', False),
 ('rank', False),
 ('plyrMgr', False)]

**Solution:** Add the `index` as an actual column

In [5]:
from dfply import mutate
managers = (managers >>
            mutate(id = managers.index))

In [6]:
managers.id.is_unique

True

In [7]:
managers.dtypes

playerID     object
yearID        int64
teamID       object
lgID         object
inseason      int64
G             int64
W             int64
L             int64
rank        float64
plyrMgr      object
id            int64
dtype: object

In [8]:
managers.shape

(3469, 11)

In [9]:
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr,id
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y,0
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y,1
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y,2
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y,3
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y,4


#### Task 2 - Create a `sqlalchemy` types `dict`

In [10]:
from sqlalchemy import String, Integer
sql_types = {'id':Integer,
             'playerID':String, 
             'plyrMgr':String,
             'teamID':String, 
             'lgID':String, 
             'yearID':Integer, 
             'inseason':Integer, 
             'G':Integer, 
             'W':Integer, 
             'L':Integer,
             'rank':Integer}

#### Task 4 - Create an `engine` and `schema`

In [11]:
!rm databases/baseball.db

In [12]:
from sqlalchemy import create_engine
mang_eng = create_engine("sqlite:///databases/baseball.db")
mang_eng.echo = True
schema = pd.io.sql.get_schema(managers, 'manager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)




#### Execute the `schema`

In [13]:
mang_eng.execute(schema)

2019-01-28 08:24:24,407 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-28 08:24:24,410 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:24:24,415 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-28 08:24:24,418 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:24:24,422 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)


2019-01-28 08:24:24,428 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:24:24,435 INFO sqlalchemy.engine.base.Engine COMMIT


#### Task 5 - Use `to_sql` with `if_exists='append'` to insert the data

In [14]:
managers.to_sql('manager', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-28 08:24:26,987 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("manager")
2019-01-28 08:24:26,989 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:24:26,995 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:24:27,060 INFO sqlalchemy.engine.base.Engine INSERT INTO manager ("playerID", "yearID", "teamID", "lgID", inseason, "G", "W", "L", rank, "plyrMgr", id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-01-28 08:24:27,061 INFO sqlalchemy.engine.base.Engine (('wrighha01', 1871, 'BS1', None, 1, 31, 20, 10, 3.0, 'Y', 0), ('woodji01', 1871, 'CH1', None, 1, 28, 19, 9, 2.0, 'Y', 1), ('paborch01', 1871, 'CL1', None, 1, 29, 10, 19, 8.0, 'Y', 2), ('lennobi01', 1871, 'FW1', None, 1, 14, 5, 9, 8.0, 'Y', 3), ('deaneha01', 1871, 'FW1', None, 2, 5, 2, 3, 8.0, 'Y', 4), ('fergubo01', 1871, 'NY2', None, 1, 33, 16, 17, 5.0, 'Y', 5), ('mcbridi01', 1871, 'PH1', None, 1, 28, 21, 7, 1.0, 'Y', 6), ('hastisc01', 1871, 'RC1', None, 1, 25, 4, 21, 9.0, 'Y', 7)  ... displaying 10 of

#### Task 6 - Query the table to make sure it all worked

In [15]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [16]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Manager = Base.classes.manager

In [17]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(Manager)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-28 08:24:29,633 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:24:29,635 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM manager
2019-01-28 08:24:29,636 INFO sqlalchemy.engine.base.Engine ()


[{'G': 31,
  'L': 10,
  'W': 20,
  'id': 0,
  'inseason': 1,
  'lgID': None,
  'playerID': 'wrighha01',
  'plyrMgr': 'Y',
  'rank': 3,
  'teamID': 'BS1',
  'yearID': 1871},
 {'G': 28,
  'L': 9,
  'W': 19,
  'id': 1,
  'inseason': 1,
  'lgID': None,
  'playerID': 'woodji01',
  'plyrMgr': 'Y',
  'rank': 2,
  'teamID': 'CH1',
  'yearID': 1871},
 {'G': 29,
  'L': 19,
  'W': 10,
  'id': 2,
  'inseason': 1,
  'lgID': None,
  'playerID': 'paborch01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'CL1',
  'yearID': 1871},
 {'G': 14,
  'L': 9,
  'W': 5,
  'id': 3,
  'inseason': 1,
  'lgID': None,
  'playerID': 'lennobi01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'FW1',
  'yearID': 1871},
 {'G': 5,
  'L': 3,
  'W': 2,
  'id': 4,
  'inseason': 2,
  'lgID': None,
  'playerID': 'deaneha01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'FW1',
  'yearID': 1871}]

## Part 2 - Awards for Managers

Now add a table for the `AwardsManagers.csv` table.

In [20]:
import pandas as pd
amanagers = pd.read_csv('~/Desktop/baseball/core/AwardsManagers.csv')
amanagers.head()

,playerID,awardID,yearID,lgID,tie,notes
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN


In [22]:
amanagers.columns

Index(['playerID', 'awardID', 'yearID', 'lgID', 'tie', 'notes'], dtype='object')

In [23]:
[(col, amanagers[col].is_unique) for col in amanagers]

[('playerID', False),
 ('awardID', False),
 ('yearID', False),
 ('lgID', False),
 ('tie', False),
 ('notes', False)]

In [24]:
from dfply import mutate
amanagers = (amanagers >>
            mutate(id = amanagers.index))

In [26]:
amanagers.id.is_unique

True

In [27]:
amanagers.dtypes

playerID    object
awardID     object
yearID       int64
lgID        object
tie         object
notes       object
id           int64
dtype: object

In [28]:
amanagers.shape

(179, 7)

In [29]:
amanagers.head()

,playerID,awardID,yearID,lgID,tie,notes,id
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN,0
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN,1
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN,2
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN,3
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN,4


In [30]:
from sqlalchemy import String, Integer
sql_types = {'id':Integer,
             'playerID': String, 
             'awardID': String, 
             'yearID': Integer, 
             'lgID': String, 
             'tie': String, 
             'notes': String}

In [31]:
!rm databases/baseball.db

In [32]:
from sqlalchemy import create_engine
mang_eng = create_engine("sqlite:///databases/baseball.db")
mang_eng.echo = True
schema = pd.io.sql.get_schema(amanagers, 'amanager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE amanager (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT amanager_pk PRIMARY KEY (id)
)




In [33]:
mang_eng.execute(schema)

2019-01-28 08:31:09,356 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-28 08:31:09,358 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:31:09,360 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-28 08:31:09,361 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:31:09,364 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE amanager (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT amanager_pk PRIMARY KEY (id)
)


2019-01-28 08:31:09,366 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:31:09,370 INFO sqlalchemy.engine.base.Engine COMMIT


In [34]:
amanagers.to_sql('amanager', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-28 08:31:33,779 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("amanager")
2019-01-28 08:31:33,781 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:31:33,785 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:31:33,789 INFO sqlalchemy.engine.base.Engine INSERT INTO amanager ("playerID", "awardID", "yearID", "lgID", tie, notes, id) VALUES (?, ?, ?, ?, ?, ?, ?)
2019-01-28 08:31:33,790 INFO sqlalchemy.engine.base.Engine (('larusto01', 'BBWAA Manager of the Year', 1983, 'AL', None, None, 0), ('lasorto01', 'BBWAA Manager of the Year', 1983, 'NL', None, None, 1), ('andersp01', 'BBWAA Manager of the Year', 1984, 'AL', None, None, 2), ('freyji99', 'BBWAA Manager of the Year', 1984, 'NL', None, None, 3), ('coxbo01', 'BBWAA Manager of the Year', 1985, 'AL', None, None, 4), ('herzowh01', 'BBWAA Manager of the Year', 1985, 'NL', None, None, 5), ('mcnamjo99', 'BBWAA Manager of the Year', 1986, 'AL', None, None, 6), ('lanieha01', 'BBWAA Manager of the Year', 1986, 'NL

In [35]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [36]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Manager = Base.classes.amanager

In [38]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(Manager)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-28 08:33:18,576 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:33:18,578 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM amanager
2019-01-28 08:33:18,580 INFO sqlalchemy.engine.base.Engine ()


[{'awardID': 'BBWAA Manager of the Year',
  'id': 0,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'larusto01',
  'tie': None,
  'yearID': 1983},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 1,
  'lgID': 'NL',
  'notes': None,
  'playerID': 'lasorto01',
  'tie': None,
  'yearID': 1983},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 2,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'andersp01',
  'tie': None,
  'yearID': 1984},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 3,
  'lgID': 'NL',
  'notes': None,
  'playerID': 'freyji99',
  'tie': None,
  'yearID': 1984},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 4,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'coxbo01',
  'tie': None,
  'yearID': 1985}]

## Part 3 - Super Hero Powers

Now make a database and table for the super hero powers.

## Problem 1
    
**Task:** One the `super_hero_powers.csv` and verify that the contents of the columns are all Boolean.  In this problem, you need to

1. Create a `dict` that defines the `pandas` column type
2. Read the file in using a `pd.read_csv`.
3. Clean up all the column labels.
    
**Be sure to write clean code!**


In [49]:
import pandas as pd
powers = pd.read_csv("./data/super_hero_powers.csv")

In [50]:
powers.head()

,hero_names,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [51]:
powers.columns

Index(['hero_names', 'Agility', 'Accelerated Healing', 'Lantern Power Ring',
       'Dimensional Awareness', 'Cold Resistance', 'Durability', 'Stealth',
       'Energy Absorption', 'Flight',
       ...
       'Web Creation', 'Reality Warping', 'Odin Force', 'Symbiote Costume',
       'Speed Force', 'Phoenix Force', 'Molecular Dissipation',
       'Vision - Cryo', 'Omnipresent', 'Omniscient'],
      dtype='object', length=168)

In [52]:
[(col, powers[col].is_unique) for col in powers]

[('hero_names', True),
 ('Agility', False),
 ('Accelerated Healing', False),
 ('Lantern Power Ring', False),
 ('Dimensional Awareness', False),
 ('Cold Resistance', False),
 ('Durability', False),
 ('Stealth', False),
 ('Energy Absorption', False),
 ('Flight', False),
 ('Danger Sense', False),
 ('Underwater breathing', False),
 ('Marksmanship', False),
 ('Weapons Master', False),
 ('Power Augmentation', False),
 ('Animal Attributes', False),
 ('Longevity', False),
 ('Intelligence', False),
 ('Super Strength', False),
 ('Cryokinesis', False),
 ('Telepathy', False),
 ('Energy Armor', False),
 ('Energy Blasts', False),
 ('Duplication', False),
 ('Size Changing', False),
 ('Density Control', False),
 ('Stamina', False),
 ('Astral Travel', False),
 ('Audio Control', False),
 ('Dexterity', False),
 ('Omnitrix', False),
 ('Super Speed', False),
 ('Possession', False),
 ('Animal Oriented Powers', False),
 ('Weapon-based Powers', False),
 ('Electrokinesis', False),
 ('Darkforce Manipulation', F

In [53]:
from dfply import *
clean_names = lambda name: name.strip().lower().replace(' ', '_')
new_names = {clean_names(old_name):old_name for old_name in powers.columns}
new_names

{'accelerated_healing': 'Accelerated Healing',
 'adaptation': 'Adaptation',
 'agility': 'Agility',
 'animal_attributes': 'Animal Attributes',
 'animal_control': 'Animal Control',
 'animal_oriented_powers': 'Animal Oriented Powers',
 'animation': 'Animation',
 'anti-gravity': 'Anti-Gravity',
 'astral_projection': 'Astral Projection',
 'astral_travel': 'Astral Travel',
 'audio_control': 'Audio Control',
 'banish': 'Banish',
 'biokinesis': 'Biokinesis',
 'camouflage': 'Camouflage',
 'changing_armor': 'Changing Armor',
 'clairvoyance': 'Clairvoyance',
 'cloaking': 'Cloaking',
 'cold_resistance': 'Cold Resistance',
 'cryokinesis': 'Cryokinesis',
 'danger_sense': 'Danger Sense',
 'darkforce_manipulation': 'Darkforce Manipulation',
 'death_touch': 'Death Touch',
 'density_control': 'Density Control',
 'dexterity': 'Dexterity',
 'dimensional_awareness': 'Dimensional Awareness',
 'dimensional_travel': 'Dimensional Travel',
 'duplication': 'Duplication',
 'durability': 'Durability',
 'echolocati

In [54]:
from dfply import rename
powers_renamed = (powers >>
              rename(**new_names))
powers_renamed.columns

Index(['hero_names', 'agility', 'accelerated_healing', 'lantern_power_ring',
       'dimensional_awareness', 'cold_resistance', 'durability', 'stealth',
       'energy_absorption', 'flight',
       ...
       'web_creation', 'reality_warping', 'odin_force', 'symbiote_costume',
       'speed_force', 'phoenix_force', 'molecular_dissipation',
       'vision_-_cryo', 'omnipresent', 'omniscient'],
      dtype='object', length=168)

In [55]:
powers_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Columns: 168 entries, hero_names to omniscient
dtypes: bool(167), object(1)
memory usage: 114.1+ KB


In [56]:
powers_renamed.dtypes

hero_names               object
agility                    bool
accelerated_healing        bool
lantern_power_ring         bool
dimensional_awareness      bool
cold_resistance            bool
durability                 bool
stealth                    bool
energy_absorption          bool
flight                     bool
danger_sense               bool
underwater_breathing       bool
marksmanship               bool
weapons_master             bool
power_augmentation         bool
animal_attributes          bool
longevity                  bool
intelligence               bool
super_strength             bool
cryokinesis                bool
telepathy                  bool
energy_armor               bool
energy_blasts              bool
duplication                bool
size_changing              bool
density_control            bool
stamina                    bool
astral_travel              bool
audio_control              bool
dexterity                  bool
                          ...  
intuitiv

In [57]:
powers_renamed.head()

,hero_names,agility,accelerated_healing,lantern_power_ring,dimensional_awareness,cold_resistance,durability,stealth,energy_absorption,flight,...,web_creation,reality_warping,odin_force,symbiote_costume,speed_force,phoenix_force,molecular_dissipation,vision_-_cryo,omnipresent,omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [58]:
def col_help(col):
    if col == 'hero_names':
        return String
    else: 
        return Boolean

In [59]:
from sqlalchemy import String, Boolean
sql_types = {column:col_help(column) for column in powers_renamed.columns}
sql_types

{'accelerated_healing': sqlalchemy.sql.sqltypes.Boolean,
 'adaptation': sqlalchemy.sql.sqltypes.Boolean,
 'agility': sqlalchemy.sql.sqltypes.Boolean,
 'animal_attributes': sqlalchemy.sql.sqltypes.Boolean,
 'animal_control': sqlalchemy.sql.sqltypes.Boolean,
 'animal_oriented_powers': sqlalchemy.sql.sqltypes.Boolean,
 'animation': sqlalchemy.sql.sqltypes.Boolean,
 'anti-gravity': sqlalchemy.sql.sqltypes.Boolean,
 'astral_projection': sqlalchemy.sql.sqltypes.Boolean,
 'astral_travel': sqlalchemy.sql.sqltypes.Boolean,
 'audio_control': sqlalchemy.sql.sqltypes.Boolean,
 'banish': sqlalchemy.sql.sqltypes.Boolean,
 'biokinesis': sqlalchemy.sql.sqltypes.Boolean,
 'camouflage': sqlalchemy.sql.sqltypes.Boolean,
 'changing_armor': sqlalchemy.sql.sqltypes.Boolean,
 'clairvoyance': sqlalchemy.sql.sqltypes.Boolean,
 'cloaking': sqlalchemy.sql.sqltypes.Boolean,
 'cold_resistance': sqlalchemy.sql.sqltypes.Boolean,
 'cryokinesis': sqlalchemy.sql.sqltypes.Boolean,
 'danger_sense': sqlalchemy.sql.sqltype

## Problem 2
    
Now define an `sqlalchemy` table for these data using `pandas` `to_sql` dataframe method.  You can use the `sqlalchemy.String` and `sqlalchemy.Boolean` columns type, which are [documented here](https://docs.sqlalchemy.org/en/latest/core/type_basics.html)

In [60]:
from sqlalchemy import create_engine
power_eng = create_engine("sqlite:///databases/powers.db")
power_eng.echo = True
schema = pd.io.sql.get_schema(powers_renamed, 'powers_renamed', keys='hero_names', con=power_eng, dtype=sql_types)
print(schema)


CREATE TABLE powers_renamed (
	hero_names VARCHAR NOT NULL, 
	agility BOOLEAN, 
	accelerated_healing BOOLEAN, 
	lantern_power_ring BOOLEAN, 
	dimensional_awareness BOOLEAN, 
	cold_resistance BOOLEAN, 
	durability BOOLEAN, 
	stealth BOOLEAN, 
	energy_absorption BOOLEAN, 
	flight BOOLEAN, 
	danger_sense BOOLEAN, 
	underwater_breathing BOOLEAN, 
	marksmanship BOOLEAN, 
	weapons_master BOOLEAN, 
	power_augmentation BOOLEAN, 
	animal_attributes BOOLEAN, 
	longevity BOOLEAN, 
	intelligence BOOLEAN, 
	super_strength BOOLEAN, 
	cryokinesis BOOLEAN, 
	telepathy BOOLEAN, 
	energy_armor BOOLEAN, 
	energy_blasts BOOLEAN, 
	duplication BOOLEAN, 
	size_changing BOOLEAN, 
	density_control BOOLEAN, 
	stamina BOOLEAN, 
	astral_travel BOOLEAN, 
	audio_control BOOLEAN, 
	dexterity BOOLEAN, 
	omnitrix BOOLEAN, 
	super_speed BOOLEAN, 
	possession BOOLEAN, 
	animal_oriented_powers BOOLEAN, 
	"weapon-based_powers" BOOLEAN, 
	electrokinesis BOOLEAN, 
	darkforce_manipulation BOOLEAN, 
	death_touch BOOLEAN, 
	

In [61]:
power_eng.execute(schema)

2019-02-02 10:33:06,744 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-02-02 10:33:06,747 INFO sqlalchemy.engine.base.Engine ()
2019-02-02 10:33:06,750 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-02-02 10:33:06,753 INFO sqlalchemy.engine.base.Engine ()
2019-02-02 10:33:06,755 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE powers_renamed (
	hero_names VARCHAR NOT NULL, 
	agility BOOLEAN, 
	accelerated_healing BOOLEAN, 
	lantern_power_ring BOOLEAN, 
	dimensional_awareness BOOLEAN, 
	cold_resistance BOOLEAN, 
	durability BOOLEAN, 
	stealth BOOLEAN, 
	energy_absorption BOOLEAN, 
	flight BOOLEAN, 
	danger_sense BOOLEAN, 
	underwater_breathing BOOLEAN, 
	marksmanship BOOLEAN, 
	weapons_master BOOLEAN, 
	power_augmentation BOOLEAN, 
	animal_attributes BOOLEAN, 
	longevity BOOLEAN, 
	intelligence BOOLEAN, 
	super_strength BOOLEAN, 
	cryokinesis BOOLEAN, 
	telepathy BOOLEAN, 
	energy_a

2019-02-02 10:33:06,758 INFO sqlalchemy.engine.base.Engine ()
2019-02-02 10:33:06,761 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) table powers_renamed already exists [SQL: '\nCREATE TABLE powers_renamed (\n\thero_names VARCHAR NOT NULL, \n\tagility BOOLEAN, \n\taccelerated_healing BOOLEAN, \n\tlantern_power_ring BOOLEAN, \n\tdimensional_awareness BOOLEAN, \n\tcold_resistance BOOLEAN, \n\tdurability BOOLEAN, \n\tstealth BOOLEAN, \n\tenergy_absorption BOOLEAN, \n\tflight BOOLEAN, \n\tdanger_sense BOOLEAN, \n\tunderwater_breathing BOOLEAN, \n\tmarksmanship BOOLEAN, \n\tweapons_master BOOLEAN, \n\tpower_augmentation BOOLEAN, \n\tanimal_attributes BOOLEAN, \n\tlongevity BOOLEAN, \n\tintelligence BOOLEAN, \n\tsuper_strength BOOLEAN, \n\tcryokinesis BOOLEAN, \n\ttelepathy BOOLEAN, \n\tenergy_armor BOOLEAN, \n\tenergy_blasts BOOLEAN, \n\tduplication BOOLEAN, \n\tsize_changing BOOLEAN, \n\tdensity_control BOOLEAN, \n\tstamina BOOLEAN, \n\tastral_travel BOOLEAN, \n\taudio_control BOOLEAN, \n\tdexterity BOOLEAN, \n\tomnitrix BOOLEAN, \n\tsuper_speed BOOLEAN, \n\tpossession BOOLEA

In [62]:
powers_renamed.to_sql('powers_renamed', 
                con=power_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-02-02 10:33:07,524 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("powers_renamed")
2019-02-02 10:33:07,526 INFO sqlalchemy.engine.base.Engine ()
2019-02-02 10:33:07,533 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-02 10:33:07,684 INFO sqlalchemy.engine.base.Engine INSERT INTO powers_renamed (hero_names, agility, accelerated_healing, lantern_power_ring, dimensional_awareness, cold_resistance, durability, stealth, energy_absorption, flight, danger_sense, underwater_breathing, marksmanship, weapons_master, power_augmentation, animal_attributes, longevity, intelligence, super_strength, cryokinesis, telepathy, energy_armor, energy_blasts, duplication, size_changing, density_control, stamina, astral_travel, audio_control, dexterity, omnitrix, super_speed, possession, animal_oriented_powers, "weapon-based_powers", electrokinesis, darkforce_manipulation, death_touch, teleportation, enhanced_senses, telekinesis, energy_beams, magic, hyperkinesis, jump, clairvoyance, d

2019-02-02 10:33:07,689 INFO sqlalchemy.engine.base.Engine ROLLBACK


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: powers_renamed.hero_names [SQL: 'INSERT INTO powers_renamed (hero_names, agility, accelerated_healing, lantern_power_ring, dimensional_awareness, cold_resistance, durability, stealth, energy_absorption, flight, danger_sense, underwater_breathing, marksmanship, weapons_master, power_augmentation, animal_attributes, longevity, intelligence, super_strength, cryokinesis, telepathy, energy_armor, energy_blasts, duplication, size_changing, density_control, stamina, astral_travel, audio_control, dexterity, omnitrix, super_speed, possession, animal_oriented_powers, "weapon-based_powers", electrokinesis, darkforce_manipulation, death_touch, teleportation, enhanced_senses, telekinesis, energy_beams, magic, hyperkinesis, jump, clairvoyance, dimensional_travel, power_sense, shapeshifting, peak_human_condition, immortality, camouflage, element_control, phasing, astral_projection, electrical_transport, fire_control, projection, summo

## Problem 3
    
Now you need to make a new `engine`, `inspect` your database, and make a `session` to query the database.

In [63]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

power_eng2 = create_engine("sqlite:///databases/powers.db") 
Session = sessionmaker(power_eng)
session = Session()

In [64]:
Base = automap_base()
Base.prepare(power_eng2, reflect=True)
Power = Base.classes.powers_renamed

In [65]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(Power)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-02-02 10:33:37,535 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-02-02 10:33:37,538 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM powers_renamed
2019-02-02 10:33:37,540 INFO sqlalchemy.engine.base.Engine ()


[{'accelerated_healing': 0,
  'adaptation': 0,
  'agility': 1,
  'animal_attributes': 0,
  'animal_control': 0,
  'animal_oriented_powers': 0,
  'animation': 0,
  'anti-gravity': 0,
  'astral_projection': 0,
  'astral_travel': 0,
  'audio_control': 0,
  'banish': 0,
  'biokinesis': 0,
  'camouflage': 0,
  'changing_armor': 0,
  'clairvoyance': 0,
  'cloaking': 0,
  'cold_resistance': 0,
  'cryokinesis': 0,
  'danger_sense': 0,
  'darkforce_manipulation': 0,
  'death_touch': 0,
  'density_control': 0,
  'dexterity': 0,
  'dimensional_awareness': 0,
  'dimensional_travel': 0,
  'duplication': 0,
  'durability': 0,
  'echolocation': 0,
  'elasticity': 0,
  'electrical_transport': 0,
  'electrokinesis': 0,
  'element_control': 0,
  'elemental_transmogrification': 0,
  'empathy': 0,
  'energy_absorption': 0,
  'energy_armor': 0,
  'energy_beams': 0,
  'energy_blasts': 0,
  'energy_constructs': 0,
  'energy_manipulation': 0,
  'energy_resistance': 0,
  'enhanced_hearing': 0,
  'enhanced_memo

## Problem 4
    
Perform `sqlalchemy` queries to answer each of the following questions.

1. How many heroes have both Super Strength and Super Speed?
2. How many heroes have names that start with the word *Black*
3. Are heroes with Agility more likely to have Stealth?
4. What fraction of all heroes that can fly also have Super Strength?
5. Consider heroes that have names that contain `"girl"`, `"boy"`, `"woman"`, or `"man"`.  Compute the following ratio

$$\frac{N(\text{boy or man})}{N(\text{girl or woman}}$$

**Hint:** You will need to use some combination of `where`, `group_by`, and `count` for each part.

In [66]:
#1 How many heroes have both Super Strength and Super Speed?
from sqlalchemy import func
stmt = select([func.count().label('both_strength_and_speed')]).\
                where(Power.super_strength == True).\
                where(Power.super_speed == True)
session.execute(stmt).fetchall() >> result_dicts

2019-02-02 10:33:42,042 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS both_strength_and_speed 
FROM powers_renamed 
WHERE powers_renamed.super_strength = 1 AND powers_renamed.super_speed = 1
2019-02-02 10:33:42,045 INFO sqlalchemy.engine.base.Engine ()


[{'both_strength_and_speed': 219}]

In [67]:
#2 How many heroes have names that start with the word Black
stmt = select([func.count().label('starts_with_black')]).\
       where(Power.hero_names.like('Black%'))
session.execute(stmt).fetchall() >> result_dicts

2019-02-02 10:33:45,541 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS starts_with_black 
FROM powers_renamed 
WHERE powers_renamed.hero_names LIKE ?
2019-02-02 10:33:45,544 INFO sqlalchemy.engine.base.Engine ('Black%',)


[{'starts_with_black': 16}]

In [68]:
#3 Are heroes with Agility more likely to have Stealth?
stmt = select([func.count(), Power.agility, Power.stealth]).\
       group_by(Power.agility).\
       group_by(Power.stealth)
result = session.execute(stmt).fetchall() >> result_dicts   
result[0]['agility']
has_agility = [(ele['count_1'], ele['stealth']) for ele in result if ele['agility']]
has_agility[1][0] / (has_agility[0][0] + has_agility[1][0])

2019-02-02 10:33:48,289 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1, powers_renamed.agility, powers_renamed.stealth 
FROM powers_renamed GROUP BY powers_renamed.agility, powers_renamed.stealth
2019-02-02 10:33:48,293 INFO sqlalchemy.engine.base.Engine ()


0.3925619834710744

In [69]:
no_agility = [(ele['count_1'], ele['stealth']) for ele in result if not ele['agility']]
no_agility[1][0] / (no_agility[0][0] + no_agility[1][0])

0.07294117647058823

In [70]:
#4 What fraction of all heroes that can fly also have Super Strength?
stmt = select([Power.flight, func.count(Power.flight).label('count_fly'), Power.super_strength]).\
       where(Power.flight == True).\
       group_by(Power.super_strength)
result = session.execute(stmt).fetchall() >> result_dicts  
can_fly = [ele['count_fly'] for ele in result]
can_fly[1] / (can_fly[0] + can_fly[1])

2019-02-02 10:33:50,898 INFO sqlalchemy.engine.base.Engine SELECT powers_renamed.flight, count(powers_renamed.flight) AS count_fly, powers_renamed.super_strength 
FROM powers_renamed 
WHERE powers_renamed.flight = 1 GROUP BY powers_renamed.super_strength
2019-02-02 10:33:50,901 INFO sqlalchemy.engine.base.Engine ()


0.6933962264150944

In [71]:
#5 Consider heroes that have names that contain "girl", "boy", "woman", or "man". Compute the following ratio
from sqlalchemy import or_
stmt = select([func.count().label('girl/woman'),Power.hero_names]).\
       where(or_(Power.hero_names.ilike('%girl%'),Power.hero_names.ilike('%woman%')))
result_bottom = session.execute(stmt).fetchall() >> result_dicts
result_bottom

2019-02-02 10:33:52,183 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS "girl/woman", powers_renamed.hero_names 
FROM powers_renamed 
WHERE lower(powers_renamed.hero_names) LIKE lower(?) OR lower(powers_renamed.hero_names) LIKE lower(?)
2019-02-02 10:33:52,186 INFO sqlalchemy.engine.base.Engine ('%girl%', '%woman%')


[{'girl/woman': 26, 'hero_names': 'Wonder Woman'}]

In [72]:
stmt = select([func.count().label('boy/man'), Power.hero_names]).\
       where(or_(Power.hero_names.ilike('%boy%'),Power.hero_names.ilike('%man%')))
result_top = session.execute(stmt).fetchall() >> result_dicts
result_top

2019-02-02 10:33:54,111 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS "boy/man", powers_renamed.hero_names 
FROM powers_renamed 
WHERE lower(powers_renamed.hero_names) LIKE lower(?) OR lower(powers_renamed.hero_names) LIKE lower(?)
2019-02-02 10:33:54,115 INFO sqlalchemy.engine.base.Engine ('%boy%', '%man%')


[{'boy/man': 62, 'hero_names': 'X-Man'}]

In [73]:
boys = [ele['boy/man'] for ele in result_top]
girls = [ele['girl/woman'] for ele in result_bottom]
ratio = boys[0] / girls[0]
ratio

2.3846153846153846

## Problem 5

Tell me another cool fact about the super powers.

In [74]:
stmt = select([Power.hero_names, Power.time_travel, Power.teleportation, Power.intelligence, Power.super_strength, Power.super_speed, Power.enhanced_memory]).\
                where(Power.time_travel == True).\
                where(Power.teleportation == True).\
                where(Power.intelligence == True).\
                where(Power.super_strength == True).\
                where(Power.super_speed == True).\
                where(Power.enhanced_memory == True)
session.execute(stmt).fetchall() >> result_dicts
# Amazo has a combination of some of the best super powers

2019-02-02 10:33:57,651 INFO sqlalchemy.engine.base.Engine SELECT powers_renamed.hero_names, powers_renamed.time_travel, powers_renamed.teleportation, powers_renamed.intelligence, powers_renamed.super_strength, powers_renamed.super_speed, powers_renamed.enhanced_memory 
FROM powers_renamed 
WHERE powers_renamed.time_travel = 1 AND powers_renamed.teleportation = 1 AND powers_renamed.intelligence = 1 AND powers_renamed.super_strength = 1 AND powers_renamed.super_speed = 1 AND powers_renamed.enhanced_memory = 1
2019-02-02 10:33:57,654 INFO sqlalchemy.engine.base.Engine ()


[{'enhanced_memory': True,
  'hero_names': 'Amazo',
  'intelligence': True,
  'super_speed': True,
  'super_strength': True,
  'teleportation': True,
  'time_travel': True}]

In [75]:
stmt = select([func.count()]).\
                where(Power.energy_blasts == True).\
                where(Power.super_strength == True)
session.execute(stmt).fetchall() >> result_dicts
# There are 100 superheros that have both super strength and energy blasts

2019-02-02 10:33:58,357 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM powers_renamed 
WHERE powers_renamed.energy_blasts = 1 AND powers_renamed.super_strength = 1
2019-02-02 10:33:58,360 INFO sqlalchemy.engine.base.Engine ()


[{'count_1': 100}]

In [76]:
stmt = select([Power.teleportation, func.count(Power.teleportation).label('count_teleportation'), Power.intelligence]).\
       where(Power.teleportation == True).\
       group_by(Power.intelligence)
result = session.execute(stmt).fetchall() >> result_dicts  
can_tele = [ele['count_teleportation'] for ele in result]
can_tele[1] / (can_tele[0] + can_tele[1])
# 1/3 of superheros that can teleport also have intelligence

2019-02-02 10:33:59,860 INFO sqlalchemy.engine.base.Engine SELECT powers_renamed.teleportation, count(powers_renamed.teleportation) AS count_teleportation, powers_renamed.intelligence 
FROM powers_renamed 
WHERE powers_renamed.teleportation = 1 GROUP BY powers_renamed.intelligence
2019-02-02 10:33:59,863 INFO sqlalchemy.engine.base.Engine ()


0.3333333333333333